### Библиотека Pandas

In [1]:
import numpy as np
import pandas as pd

### Объекты класса pandas.Series
Серия - это массив NumPy, но с именем и дополнительной индексацией ячеек:
индексами на основе меток (label-based indexing).

Если индексы-метки не указать при создании серии, то по умолчанию они совпадают с порядковыми индексами от 0.

In [2]:
ages = pd.Series([22, 35, 25, 16], name='age')
print(ages)

0    22
1    35
2    25
3    16
Name: age, dtype: int64


Можно явно указать индексы через параметр конструктора index:

In [4]:
named_ages = pd.Series(data=[22, 35, 25, 16], index=['Aline', 'Beatrice', 'Clara', 'Diana'])
print(named_ages)

Aline       22
Beatrice    35
Clara       25
Diana       16
dtype: int64


In [5]:
named_ages['Beatrice']

35

In [6]:
named_ages[1]

35

### Сложности двойной индексации и их решение: loc, iloc, at, iat

Создадим простую серию с числами и возьмём её срез:

In [10]:
numbers = pd.Series(range(0, 1000, 100))
the_slice = numbers[2:7]
print(the_slice)

2    200
3    300
4    400
5    500
6    600
dtype: int64


Обратите внимание, что после среза индексы - метки остались закреплёнными за своими значениями элементов. Это правильно с точки зрения идентификации хранимых данных. Особенно, если данные про одну сущность будут находится в разных сериях и потом их нужно будет сопоставлять.

Однако, в результате появляется неоднозначность индексации.

Доступ с квадратными скобками теперь работает только по индексам-меткам:

In [12]:
the_slice[2]

200

если же для такой неоднозначной ситуации взять срез, то мы получим "жёлтую карточку" - предупреждение от Pandas:

In [16]:
the_slice

2    200
3    300
4    400
5    500
6    600
dtype: int64

Как видите, мы получили срез по порядковым индексам, но библиотека предупреждает о будущем изменении поведения: скоро в таких ситуациях будет осуществляться срез по индексам-меткам.

Достоверно доступиться к элементам серии по порядковаму индексу можно через локатор iloc:

In [30]:
the_slice.iloc[0:3]

2    200
3    300
4    400
dtype: int64

In [29]:
the_slice

2    200
3    300
4    400
5    500
6    600
dtype: int64

Элемент с индексом-меткой 5 включён в срез, что противоречит логике range(start, stop, step). Почему так? Подумайте про срезы с метками строкового типа и посмотрете на пример ниже:

In [32]:
named_ages['Beatrice': 'Diana']

Beatrice    35
Clara       25
Diana       16
dtype: int64

Если доступ нужен только к одному элементу, то можно вместо loc и iloc воспользоваться at и iat:

In [33]:
the_slice.at[4]

400

In [34]:
the_slice.iat[4]

600

In [44]:
the_slice.loc[2:4]

2    200
3    300
4    400
dtype: int64

In [45]:
the_slice[0:2]

2    200
3    300
dtype: int64

### Добавление данных в серию, удаление

В отличие от массивов NumPy, серии имеют переменный размер:

In [46]:
s = pd.Series(data=[0, 10, 20, 30], index=[1, 2, 3, 4])
print(s)

1     0
2    10
3    20
4    30
dtype: int64


In [47]:
s.pop(1) # Удаление элемета с индексом-меткой 1

0

In [48]:
print(s)

2    10
3    20
4    30
dtype: int64


Добавление элементов с присваиванием индекса-метки можно делать через локатор loc:

In [49]:
s.loc[5] = 50

In [50]:
s

2    10
3    20
4    30
5    50
dtype: int64

Обратите внимание, что таким добавлением можно нарушить порядок индексов:

In [51]:
s.loc[4] = 40

In [52]:
s

2    10
3    20
4    40
5    50
dtype: int64

In [53]:
s.loc[1] = 100

In [54]:
s

2     10
3     20
4     40
5     50
1    100
dtype: int64

Если индексы-метки нужно пересортировать, можно сделать это при помощи sort_index:

In [55]:
s.sort_index(inplace=True)

In [56]:
print(s)

1    100
2     10
3     20
4     40
5     50
dtype: int64


Ещё более куръёзную ситуацию с индексами можно создать при помощи конкатенации серий функцией pd.concat:

In [58]:
a = pd.Series(data=[10, 20, 40], index=[1, 2, 4])
b = pd.Series(data=[100, 200, 300], index=[1, 2, 3])
c = pd.concat([a, b])
print(c)

1     10
2     20
4     40
1    100
2    200
3    300
dtype: int64


In [59]:
c.index

Int64Index([1, 2, 4, 1, 2, 3], dtype='int64')

In [61]:
c[1] = 10000
print(c)

1    10000
2       20
4       40
1    10000
2      200
3      300
dtype: int64


Итак, индексы-метки могут оказаться не уникальными!!! И присваивание в такую метку приводит к изменению всех значений с этой меткой.

Лучше такой ситуации не создавать. Вот так можно проверить, что метки уникальны:

In [62]:
c.index.is_unique

False

In [66]:
c = c.reset_index()[0]

In [67]:
c

0    10000
1       20
2       40
3    10000
4      200
5      300
Name: 0, dtype: int64

### Редукция серий

Агрегирующие методы по сравнению с массивами NumPy.ndarray в сериях предопределены: в них допускаются и просто игнорируются пустые ячейки, которые представоены как NaN (Not a Number).

In [69]:
values = [1, 3, 5, np.nan, 1, np.nan, 3]
array = np.array(values)
series = pd.Series(values)

for container in array, series:
    print(type(container))
    print(container.min(), container.max(), container.sum())

<class 'numpy.ndarray'>
nan nan nan
<class 'pandas.core.series.Series'>
1.0 5.0 13.0


In [70]:
series.unique()

array([ 1.,  3.,  5., nan])

In [71]:
series.mean()

2.6

In [73]:
series.value_counts()

1.0    2
3.0    2
5.0    1
dtype: int64

In [1]:
import pandas as pd
import numpy as np

### Векторные (массовые) рперации
Как и для массивов NumPy для серий работают массовые операции:

In [2]:
A = pd.Series(data=[100, 20, 30, 40], index=["A", "B",  "C", "D"])
B = pd.Series(data=[1, 2, 3, 4], index=["A", "B", "C", "D"])

In [3]:
A + B

A    101
B     22
C     33
D     44
dtype: int64

In [4]:
-A + 25

A   -75
B     5
C    -5
D   -15
dtype: int64

In [5]:
A > 20

A     True
B    False
C     True
D     True
dtype: bool

In [6]:
B != 3

A     True
B     True
C    False
D     True
dtype: bool

In [7]:
B == 3

A    False
B    False
C     True
D    False
dtype: bool

In [8]:
B.apply(lambda x: x*100)

A    100
B    200
C    300
D    400
dtype: int64

Значение из одной серии можно массово чкопировать в другую серию методом update, при этом опора будет производится на индексы-метки, а не на последовательность элементов. При этом для меток старой серии, к которой не найдены метки новой серии, будут оставлены старые значения:

In [23]:
A = pd.Series(data=[100, 200, 500, 0, 4], index=['Aline', 'Beatrice', 'Clara', 'Bob', 'Diana'])

In [24]:
B = pd.Series(data=[300, 19873], index=['Diana', 'Aline'])

In [25]:
print(A)

Aline       100
Beatrice    200
Clara       500
Bob           0
Diana         4
dtype: int64


In [26]:
print(B)

Diana      300
Aline    19873
dtype: int64


In [27]:
A.update(B)

In [28]:
print(A)

Aline       19873
Beatrice      200
Clara         500
Bob             0
Diana         300
dtype: int64


### Объекты класса DataFrame

После знакомства с сериями Series легче понять как формируется лист с данными. По сути это несколько серий-столбцов с одинаковым индексом по вертикали, а последовательность имён столбцов составляет горизонтальный тндекс таблицы.

In [29]:
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]}, index=['first', 'second', 'third'])

In [30]:
df

,A,B,C
first,1,4,7
second,2,5,8
third,3,6,9


In [31]:
df.index

Index(['first', 'second', 'third'], dtype='object')

In [32]:
df.columns

Index(['A', 'B', 'C'], dtype='object')

Доступ к сериям - столбцам можно делать через квадратные скобки или через оператор . (если имя является допустимым идентификатором):

In [33]:
df['A']

first     1
second    2
third     3
Name: A, dtype: int64

In [34]:
df.A

first     1
second    2
third     3
Name: A, dtype: int64

In [35]:
type(df.A)

pandas.core.series.Series

In [36]:
df['D'] = df.A*100 + df.B*10 + df.C
df.D

first     147
second    258
third     369
Name: D, dtype: int64

In [37]:
df

,A,B,C,D
first,1,4,7,147
second,2,5,8,258
third,3,6,9,369


In [41]:
df['G'] = (df.A < 3) & (df.D > 150)

In [42]:
df

,A,B,C,D,G
first,1,4,7,147,False
second,2,5,8,258,True
third,3,6,9,369,False


In [43]:
df['D']

first     147
second    258
third     369
Name: D, dtype: int64

### Сложности индексации и их решение: дoc, iloc

Квадратные скобки  дают доступ к столбцам, так и, в случае среза - к строкам:

In [45]:
df[1:3] # Срез по порядковым индексам

,A,B,C,D,G
second,2,5,8,258,True
third,3,6,9,369,False


In [47]:
df['first': 'second'] # срез по индексам-меткам (включая конечную)

,A,B,C,D,G
first,1,4,7,147,False
second,2,5,8,258,True


Как вы понимаете, это может приводить к неоднозначности, поэтому лучше использовать loc и iloc:

In [50]:
df.iloc[1:3]

,A,B,C,D,G
second,2,5,8,258,True
third,3,6,9,369,False


In [51]:
df.loc['first': 'second']

,A,B,C,D,G
first,1,4,7,147,False
second,2,5,8,258,True


In [53]:
df.loc['second']

A       2
B       5
C       8
D     258
G    True
Name: second, dtype: object

Запятая в квадратных скобках позволяет доставать значение конкретной ячейки
или срез по строкам и столбцам одновременно

In [54]:
df.loc['first', 'A']

1

In [55]:
df

,A,B,C,D,G
first,1,4,7,147,False
second,2,5,8,258,True
third,3,6,9,369,False


In [57]:
df.loc['first', 'A']

1

In [58]:
df.loc['first': 'second', 'B':'C']

,B,C
first,4,7
second,5,8


In [59]:
df.loc[['third', 'first'],['D', 'B', 'C']]

,D,B,C
third,369,6,9
first,147,4,7


In [60]:
df.iloc[:, ::-1]

,G,D,C,B,A
first,False,147,7,4,1
second,True,258,8,5,2
third,False,369,9,6,3


In [61]:
df.loc[['third', 'first'], ['D', 'B', 'C']]

,D,B,C
third,369,6,9
first,147,4,7


In [62]:
df.sum(axis=1)

first     159.0
second    274.0
third     387.0
dtype: float64

In [63]:
df.max()

A       3
B       6
C       9
D     369
G    True
dtype: object

In [64]:
df.max()

A       3
B       6
C       9
D     369
G    True
dtype: object

In [66]:
df.idxmax()

A     third
B     third
C     third
D     third
G    second
dtype: object

In [69]:
df.aggregate(['min', 'idxmax', 'max', 'mean', 'sum'])

,A,B,C,D,G
min,1,4,7,147,False
idxmax,third,third,third,third,second
max,3,6,9,369,True
mean,2.0,5.0,8.0,258.0,0.333333
sum,6,15,24,774,1


In [70]:
df

,A,B,C,D,G
first,1,4,7,147,False
second,2,5,8,258,True
third,3,6,9,369,False


In [73]:
df2 = df.drop(columns = ['G', 'G'])

In [72]:
df

,A,B,C,D,G
first,1,4,7,147,False
second,2,5,8,258,True
third,3,6,9,369,False


In [74]:
df2

,A,B,C,D
first,1,4,7,147
second,2,5,8,258
third,3,6,9,369


In [76]:
df.drop(labels=['first'])

,A,B,C,D,G
second,2,5,8,258,True
third,3,6,9,369,False


In [77]:
df.loc['second', 'C'] = np.nan

In [78]:
df

,A,B,C,D,G
first,1,4,7.0,147,False
second,2,5,NaN,258,True
third,3,6,9.0,369,False


In [79]:
df.dropna() # удалить все строки, где есть nan

,A,B,C,D,G
first,1,4,7.0,147,False
third,3,6,9.0,369,False


In [80]:
df.dropna(axis='columns') # можно вместо строк с Nan выбросить столбцы

,A,B,D,G
first,1,4,147,False
second,2,5,258,True
third,3,6,369,False


In [81]:
df.fillna(0)

,A,B,C,D,G
first,1,4,7.0,147,False
second,2,5,0.0,258,True
third,3,6,9.0,369,False


In [82]:
df.interpolate()

,A,B,C,D,G
first,1,4,7.0,147,False
second,2,5,8.0,258,True
third,3,6,9.0,369,False


In [83]:
commits = pd.DataFrame({"name": ["Петя", "Вася", "Таня", "Петя", "Вася", "Таня", "Вася", "Петя", "Таня"], 
                       "potatoes": [3, 4, 0, 3, 4, 0, 6, 3, 0],
                       "carrots": [0, 2, 5, 2, 1, 4, 15, 2, 6]})

In [84]:
commits

,name,potatoes,carrots
0,Петя,3,0
1,Вася,4,2
2,Таня,0,5
3,Петя,3,2
4,Вася,4,1
5,Таня,0,4
6,Вася,6,15
7,Петя,3,2
8,Таня,0,6


In [85]:
commits.sum()

name        ПетяВасяТаняПетяВасяТаняВасяПетяТаня
potatoes                                      23
carrots                                       37
dtype: object

In [87]:
commits.agg(['min', 'max'])

,name,potatoes,carrots
min,Вася,0,0
max,Таня,6,15


In [89]:
commits.groupby("name")

In [90]:
commits

,name,potatoes,carrots
0,Петя,3,0
1,Вася,4,2
2,Таня,0,5
3,Петя,3,2
4,Вася,4,1
5,Таня,0,4
6,Вася,6,15
7,Петя,3,2
8,Таня,0,6


In [92]:
commits.groupby('name').sum() # количество овощей, "сгрупированное" по имени

,potatoes,carrots
name,,
Вася,14,18
Петя,9,4
Таня,0,15


In [93]:
commits.groupby('name').carrots.aggregate(['min', 'max'])

,min,max
name,,
Вася,1,15
Петя,0,2
Таня,4,6


In [94]:
commits.groupby('name').aggregate(['min', 'max'])

potatoes     carrots    
          min max     min max
name                         
Вася        4   6       1  15
Петя        3   3       0   2
Таня        0   0       4   6